In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms, models
import os
import time
import copy

import random
from torch.utils import data

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:

data_path = r"/content/drive/My Drive/cifar10/data"

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) # mean and std from resnet18
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]))


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class_names = ['airplane', 'bird']
label_map = {0: 0, 2: 1}

cifar2_train = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]
              
nt = len(cifar2_train) 
nv = len(cifar2_val)
n_train = int(400)  
n_val = int(100)
idxT = list(range(nt)) 
idxV = list(range(nv))  
random.shuffle(idxT)  
random.shuffle(idxV)

train_idx = idxT[:n_train]
val_idx = idxV[:n_val]

In [ ]:
image_datasets_raw = {'train': cifar2_train, 'val': cifar2_val}
image_datasets = {x: data.Subset(image_datasets_raw[x], train_idx if x == 'train' else val_idx) for x in image_datasets_raw }


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
print(f"Training on device {device}.")
use_cuda = torch.cuda.is_available()

Training on device cuda.


In [ ]:
image_datasets['train'], image_datasets['val']

(<torch.utils.data.dataset.Subset at 0x7ff594985da0>,
 <torch.utils.data.dataset.Subset at 0x7ff594985dd8>)

In [ ]:
def train_model(model, loss_fn, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

               #forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = loss_fn(outputs, labels)

                    # backwards pass
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                #stats
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print('\n')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load weights of best model
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_all_params_opt = models.resnet18(pretrained=True)
num_ftrs = model_all_params_opt.fc.in_features #number of outputs of linear layer
model_all_params_opt.fc = nn.Linear(num_ftrs, len(class_names)) # sets last layer of model

model_all_params_opt = model_all_params_opt.to(device) # sends model to device

loss_fn = nn.CrossEntropyLoss()

# optimizing on all parameters
optimizer = optim.SGD(model_all_params_opt.parameters(), lr=0.001, momentum=0.9)
#decay lr by a factor of gamme each 7 steps(epochs)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
model_all_params_opt = train_model(model = model_all_params_opt, loss_fn = loss_fn, optimizer = optimizer, scheduler = scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6361 Acc: 0.6850
val Loss: 0.7812 Acc: 0.6600


Epoch 1/24
----------
train Loss: 0.5260 Acc: 0.7800
val Loss: 0.2427 Acc: 0.9200


Epoch 2/24
----------
train Loss: 0.6198 Acc: 0.7950
val Loss: 0.4674 Acc: 0.8300


Epoch 3/24
----------
train Loss: 0.5608 Acc: 0.7700
val Loss: 0.2431 Acc: 0.9100


Epoch 4/24
----------
train Loss: 0.4552 Acc: 0.8350
val Loss: 0.3337 Acc: 0.8700


Epoch 5/24
----------
train Loss: 0.5513 Acc: 0.8025
val Loss: 0.2492 Acc: 0.9000


Epoch 6/24
----------
train Loss: 0.3678 Acc: 0.8475
val Loss: 0.2008 Acc: 0.9300


Epoch 7/24
----------
train Loss: 0.3100 Acc: 0.8700
val Loss: 0.1987 Acc: 0.9500


Epoch 8/24
----------
train Loss: 0.2350 Acc: 0.9125
val Loss: 0.1629 Acc: 0.9600


Epoch 9/24
----------
train Loss: 0.2958 Acc: 0.8850
val Loss: 0.1830 Acc: 0.9300


Epoch 10/24
----------
train Loss: 0.2802 Acc: 0.8850
val Loss: 0.1472 Acc: 0.9700


Epoch 11/24
----------
train Loss: 0.2110 Acc: 0.9250
val Loss: 0.1549 Acc:

Finetunign the model
Load a pretrained model and reset final fully connected layer.

This version only optimizes on the last layer

Here, we need to freeze all the network except the final layer. We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward().

In [ ]:
model_ll = torchvision.models.resnet18(pretrained=True)
for param in model_ll.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_ll.fc.in_features
model_ll.fc = nn.Linear(num_ftrs, len(class_names))

model_ll = model_ll.to(device)

loss_fn = nn.CrossEntropyLoss()

# only parameters of final layer are being optimized
optimizer = optim.SGD(model_ll.fc.parameters(), lr=0.001, momentum=0.9)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model_all_params_opt = train_model(model = model_ll, loss_fn = loss_fn, optimizer = optimizer, scheduler = scheduler, num_epochs=25)


Epoch 0/24
----------
train Loss: 0.5786 Acc: 0.6975
val Loss: 0.2529 Acc: 0.8900


Epoch 1/24
----------
train Loss: 0.6184 Acc: 0.7600
val Loss: 0.2544 Acc: 0.9000


Epoch 2/24
----------
train Loss: 0.6262 Acc: 0.7400
val Loss: 0.2077 Acc: 0.9000


Epoch 3/24
----------
train Loss: 0.5006 Acc: 0.7725
val Loss: 0.2186 Acc: 0.9200


Epoch 4/24
----------
train Loss: 0.4907 Acc: 0.7925
val Loss: 0.2075 Acc: 0.9300


Epoch 5/24
----------
train Loss: 0.4746 Acc: 0.8125
val Loss: 0.1556 Acc: 0.9300


Epoch 6/24
----------
train Loss: 0.5502 Acc: 0.7800
val Loss: 0.3973 Acc: 0.8500


Epoch 7/24
----------
train Loss: 0.3724 Acc: 0.8450
val Loss: 0.1641 Acc: 0.9300


Epoch 8/24
----------
train Loss: 0.3388 Acc: 0.8525
val Loss: 0.1330 Acc: 0.9400


Epoch 9/24
----------
train Loss: 0.3394 Acc: 0.8450
val Loss: 0.1323 Acc: 0.9500


Epoch 10/24
----------
train Loss: 0.3520 Acc: 0.8575
val Loss: 0.1271 Acc: 0.9500


Epoch 11/24
----------
train Loss: 0.3029 Acc: 0.8675
val Loss: 0.1569 Acc: